### 1. 학습, 시험 테이터 셋 나누기

In [1]:
import tensorflow as tf

training_dir = 'imgs/face_training'
validation_dir = 'imgs/face_validation'

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    # Inception V3 입력 크기 150*150
    target_size=(150, 150),
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    # Inception V3 입력 크기 150*150
    target_size=(150, 150),
    class_mode='categorical')


Found 316 images belonging to 4 classes.
Found 76 images belonging to 4 classes.


### inceptionV3 모델 불러오기

In [2]:
# import urllib


# weights_url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# weights_file = 'models/inception_v3.h5'

# # weights_url에서 파일을 가져와 weights_file이라는 이름으로 저장
# urllib.request.urlretrieve(weights_url, weights_file)

weights_file = 'models/inception_v3.h5'

# Inception V3 신경망 모델 생성
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None)

# 생성된 모델에 가져온 가중치 부여
pre_trained_model.load_weights(weights_file)

In [ ]:
# 모델 정보 출력
pre_trained_model.summary()

In [3]:
# 가져온 신경망의 parameter가 훈련되지 않도록 동결
for layer in pre_trained_model.layers:
    layer.trainable = False

# mixed7 층의 마지막 출력을 가리키는 변수 생성
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

# 출력 펼치기
x = tf.keras.layers.Flatten()(last_output)
# Dense 층 추가
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# 다중 분류를 위해 소프트맥스 함수를 사용한 출력층 생성
x = tf.keras.layers.Dense(4, activation='softmax')(x)

# 모델 생성
model = tf.keras.Model(pre_trained_model.input, x)

# 훈련 방법 설정
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['acc'])

# 훈련 시작
model.fit(train_generator, epochs=40, validation_data=validation_generator)

Epoch 1/40


c:\Users\rnehd\.conda\envs\cap\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 424ms/step - acc: 0.2594 - loss: 3.0736 - val_acc: 0.5263 - val_loss: 1.1863
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 304ms/step - acc: 0.4254 - loss: 1.2613 - val_acc: 0.5000 - val_loss: 0.9849
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 301ms/step - acc: 0.5073 - loss: 1.0808 - val_acc: 0.7105 - val_loss: 0.8018
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 299ms/step - acc: 0.6510 - loss: 0.9019 - val_acc: 0.6579 - val_loss: 0.7953
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - acc: 0.6707 - loss: 0.8388 - val_acc: 0.4737 - val_loss: 1.0021
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 296ms/step - acc: 0.5787 - loss: 0.9668 - val_acc: 0.7237 - val_loss: 0.6670
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - acc: 0.7233 - loss: 0.6947 - val_acc: 0.7105 - val_loss: 0.9335
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 284ms/step - acc: 0.6676 - loss: 0.8917 - val_acc: 0.6711 - val_loss: 0.7944
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - acc: 0.6781 -

In [6]:
model.save('models/inceptionV3_celebrity.h5')

In [7]:
import cv2
import numpy as np

# 이미지 로드
image = cv2.imread('imgs/predict/GuDongBin_faces/bin.jpg')

# # 이미지 크기 조정 (150, 150)으로 조정
# image = cv2.resize(image, (150, 150))

# 모델이 기대하는 형태로 차원 추가 (batch 차원 추가)
input_image = np.expand_dims(image, axis=0)

# 이미지를 0과 1사이의 값으로 조정 (정규화)
input_image = input_image/255.0

# 입력 이미지에 대한 예측
model.predict(input_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[6.5434474e-04, 1.0179839e-01, 1.4838825e-01, 7.4915904e-01]],
      dtype=float32)

In [8]:
pred = model.predict(input_image)

# 학습시켰던 연예인 이름 레이블
class_labels = ['이승기', '남주혁', '박보영', '서강준']

# 가장 큰 원소의 인덱스를 반환
predict_class_index = np.argmax(pred)

# 가장 높은 확률을 가진 클래스의 레이블명 반환
predict_label = class_labels[predict_class_index]

print(f'당신이 {predict_label}일 확률은 {round(pred[0][predict_class_index]*100, 2)}% 입니다!')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
당신이 서강준일 확률은 14.84% 입니다!
